# Install Required Libraries

In [2]:
!pip install openai requests

# Set API Keys

In [ ]:
OPENAI_API_KEY = "your_openai_api_key_here"
OPENWEATHER_API_KEY = "your_openweathermap_api_key_here"


# Import Libraries

In [4]:
import requests
import openai
openai.api_key = OPENAI_API_KEY

# Define Planner Agent

In [5]:
class PlannerAgent:
    def __init__(self, goal):
        self.goal = goal
        self.plan = ["spacex", "weather", "summarize"]

    def get_plan(self):
        return self.plan


# Define SpaceX Agent

In [6]:
class SpaceXAgent:
    def run(self):
        res = requests.get("https://api.spacexdata.com/v4/launches/next").json()
        location_id = res['launchpad']
        launch_time = res['date_utc']

        site_data = requests.get(f"https://api.spacexdata.com/v4/launchpads/{location_id}").json()

        return {
            'launch_time': launch_time,
            'location': {
                'name': site_data['name'],
                'lat': site_data['latitude'],
                'lon': site_data['longitude']
            }
        }


# Define Weather Agent

In [14]:
class WeatherAgent:
    def __init__(self, api_key):
        self.api_key = api_key

    def run(self, location, launch_time):
        lat, lon = location['lat'], location['lon']
        print(f" Checking weather for: {location['name']} (Lat: {lat}, Lon: {lon})")

        url = f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={self.api_key}&units=metric"
        print(f" Weather API URL: {url}")

        res = requests.get(url)
        data = res.json()

        if 'weather' not in data:
            print(" ERROR: 'weather' key not found.")
            print(" Response:", data)
            return {
                'description': 'unknown',
                'wind_speed': 0,
                'visibility': 0,
                'clouds': 0
            }

        return {
            'description': data['weather'][0]['description'],
            'wind_speed': data['wind']['speed'],
            'visibility': data.get('visibility', 0),
            'clouds': data.get('clouds', {}).get('all', 0)
        }


# Define Summarizer Agent

In [15]:
class SummarizerAgent:
    def run(self, weather_data, location_name, launch_time):
        wind = weather_data['wind_speed']
        visibility = weather_data['visibility']
        clouds = weather_data['clouds']
        desc = weather_data['description']

        issues = []

        if wind > 10:
            issues.append("high wind")
        if visibility < 5000:
            issues.append("low visibility")
        if clouds > 70:
            issues.append("heavy cloud cover")

        if issues:
            reason = ", ".join(issues)
            return f" Launch at {location_name} on {launch_time} may be delayed due to {reason} ({desc})."
        else:
            return f" Launch at {location_name} on {launch_time} is likely on schedule. Weather looks good ({desc})."


# Run the Entire Agent System

In [13]:
goal = "Find the next SpaceX launch, check weather at that location, then summarize if it may be delayed."
# Pass the 'goal' argument to the PlannerAgent constructor
planner = PlannerAgent(goal)
plan = planner.get_plan() # Assuming get_plan() is the correct method name based on the PlannerAgent definition
print("📋 Plan from Planner:", plan)

# Fetch SpaceX launch
spacex = SpaceXAgent()
launch_data = spacex.run()
print("🚀 Launch Info:", launch_data)

# Ensure OPENWEATHER_API_KEY is defined in a previous cell
weather = WeatherAgent(OPENWEATHER_API_KEY)
weather_data = weather.run(launch_data['location'], launch_data['launch_time'])
print("🌤️ Weather Data:", weather_data)

# Ensure OPENAI_API_KEY is defined in a previous cell
summarizer = SummarizerAgent() # SummarizerAgent does not require API key in __init__ based on current definition
summary = summarizer.run(weather_data, launch_data['location']['name'], launch_data['launch_time'])

# Final Output
print("\n🔎 FINAL SUMMARY:\n", summary)

📋 Plan from Planner: ['spacex', 'weather', 'summarize']
🚀 Launch Info: {'launch_time': '2022-11-01T13:41:00.000Z', 'location': {'name': 'KSC LC 39A', 'lat': 28.6080585, 'lon': -80.6039558}}
📍 Checking weather for: KSC LC 39A (Lat: 28.6080585, Lon: -80.6039558)
🌐 Weather API URL: https://api.openweathermap.org/data/2.5/weather?lat=28.6080585&lon=-80.6039558&appid=e2fd32f1deba0073258ead956d5fe5b9&units=metric
🌤️ Weather Data: {'description': 'overcast clouds', 'wind_speed': 7.2, 'visibility': 10000, 'clouds': 100}

🔎 FINAL SUMMARY:
 🚫 Launch at KSC LC 39A on 2022-11-01T13:41:00.000Z may be delayed due to heavy cloud cover (overcast clouds).
